In [84]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import trimesh
from skimage import measure
import meshplot as mp
from torch.utils.data import DataLoader, Dataset
import os
import time
from datetime import timedelta, datetime
import random
import math
from tools import * 

### Run denoising 

In [85]:
def run_denoising(diffusion_model_dt, 
                  diffusion_model_idx, 
                  shape_idx=94, # shape index to add noise and denoise (caution:  note that shape_idx != shape_indices_from_decoder[shape_idx])
                  timestep = 30000, # pick timestep to add noise
                  save_denoised=True):

    # load diffusion log and the number of shpaes used for diffusion (n_shapes) and the the decoder model used (decoder_model)
    log_filename = f'./diffusion logs & models/{diffusion_model_dt}/{diffusion_model_dt}.log'
    with open(log_filename) as f:
        f = f.readlines()
        
    decoder_model_type = "SGI" # default value as some earlier logs are missing with this
    beta_schedule = "linear" # default is set as some earlier logs are missing with this
    for line in f: 
        if "Decoder model type" in line:
            decoder_model_type = "SGI"#line.split('=')[1].strip()
        if "# of shapes used for diffusion" in line :
            n_shapes = 6278#int(line.split('=')[1].strip())
        if "Decoder model used" in line :
            decoder_model = "11012022_133638"#line.split('=')[1].strip()
        if "Beta schedule" in line:
            beta_schedule = 'linear'#line.split('=')[1].strip()
        if "# of steps for diffusion forward process" in line:
            forward_process_t = 30000#line.split('=')[1].strip()
    betas = make_beta_schedule(schedule=beta_schedule, n_timesteps=int(forward_process_t))

    # load diffusion model
    model_filename = f'./diffusion logs & models/{diffusion_model_dt}/conditional model_SGI_{diffusion_model_dt}_{diffusion_model_idx}'
    model = ConditionalModel(forward_process_t)
    model.load_state_dict(torch.load(model_filename))
    model.eval()

    # load the decoder and shapecodes
    if decoder_model_type == "SGI":
        decoder, _ = get_decoder(decoder_model, decoder_epoch="latest",parallel=True) 
        shapecode, _, shape_indices_from_decoder = get_shapecode(decoder_model, n_shapes=n_shapes, shapecode_epoch="latest") 
    elif decoder_model_type == "deep_sdf":
        deocoder, _ = get_decoder(experiment_directory=f"../DeepSDF2/examples/{model_datetime}", checkpoint="latest")
        shapecode, shapecode_epoch = get_deep_sdf_shapecodes(experiment_directory=f"../DeepSDF2/examples/{model_datetime}", checkpoint="latest")

    # preprocess shapecode
    std = np.std(np.asarray(shapecode))
    mean = np.mean(np.asarray(shapecode))
    shapecode = (shapecode-mean)/std

    # retrieve original shapecode corresponding to the shape_idx
    original_code=std*shapecode[shape_idx]+mean
        
    # add noise to the original shapecode 
    #noised_code = std*q_sample(shapecode[shape_idx], torch.tensor([timestep-1]), betas)+mean
    noised_code = q_sample(shapecode[shape_idx], torch.tensor([timestep-1]), betas)
    
    # denoise the noised code 
    denoised_code = noised_code
    #inters=[std*denoised_code+mean]
    for i in reversed(range(int(timestep)-1)):
        #if i%100==0:
        #    inters.append(std*denoised_code+mean)
        denoised_code = p_sample(model, denoised_code, i, betas)
    #inters.append(std*denoised_code+mean)
    denoised_code = std*denoised_code+mean
    
    # save the denoise shape 
    if save_denoised: 
        denoising = datetime.now() 
        denoising = denoising.strftime("%m%d%Y_%H%M%S")

        if not os.path.isdir(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{shape_idx}'):
            os.mkdir(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{shape_idx}')
            print(f'created directory: ./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{shape_idx}')

        np.savez(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{shape_idx}/{denoising}', 
                 original_code=original_code,
                 noised_code=noised_code,
                 denoised_code=denoised_code.detach().numpy())
    
    return decoder, original_code, noised_code, denoised_code#, inters

In [87]:
for i in range(1):
    t = time.time()
    decoder, original_code, noised_code, denoised_code = run_denoising('11082022_135018', 8000, shape_idx=12, timestep = 30000, save_denoised = False)
    denoised_verts, faces = reconstruct_normalized_shape(decoder, denoised_code, N=100, max_batch = 10000)
    print(time.time()-t)
    mp.plot(denoised_verts, faces)

linear
53.135552167892456


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.030103…

In [77]:
np.savez('vary_5',faces=faces, verts=denoised_verts)

In [82]:
import time

In [83]:
for i in range(5):
    
    decoder, original_code, noised_code, denoised_code = run_denoising('10062022_011812', 6100, shape_idx=2, timestep = 30000, save_denoised = False)
    denoised_verts, faces = reconstruct_normalized_shape(decoder, denoised_code, N=100, max_batch = 10000)
    
    mp.plot(denoised_verts, faces)

linear


FileNotFoundError: [Errno 2] No such file or directory: './diffusion logs & models/10062022_011812/conditional model_SGI_10062022_011812_6100'

In [195]:
for i in range(5):
    decoder, original_code, noised_code, denoised_code = run_denoising('10062022_011812', 6100, shape_idx=2, timestep = 1, save_denoised = False)
    denoised_verts, faces = reconstruct_normalized_shape(decoder, denoised_code, N=100, max_batch = 10000)
    mp.plot(denoised_verts, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.0802021…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.2069940…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.3902783…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.7181038…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.0875205…

In [193]:
for i in range(5):
    decoder, original_code, noised_code, denoised_code = run_denoising('10062022_011812', 6100, shape_idx=94, timestep = 2000, save_denoised = False)
    denoised_verts, faces = reconstruct_normalized_shape(decoder, denoised_code, N=100, max_batch = 10000)
    mp.plot(denoised_verts, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.011925…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.011802…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.004834…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.007824…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.013775…

In [194]:
for i in range(5):
    decoder, original_code, noised_code, denoised_code = run_denoising('10062022_011812', 6100, shape_idx=534, timestep = 3000, save_denoised = False)
    denoised_verts, faces = reconstruct_normalized_shape(decoder, denoised_code, N=100, max_batch = 10000)
    mp.plot(denoised_verts, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.026057…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.014058…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.015756…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.004558…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.033564…

In [198]:
for i in range(5):
    decoder, original_code, noised_code, denoised_code = run_denoising('10062022_011812', 6100, shape_idx=534, timestep = 7000, save_denoised = False)
    denoised_verts, faces = reconstruct_normalized_shape(decoder, denoised_code, N=100, max_batch = 10000)
    mp.plot(denoised_verts, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.012328…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001438…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.004428…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.014010…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.484023…

### Compare denoised sdfs & original decoder sdfs

In [27]:
run_denoising('10062022_011812', 6100, print_original=True, print_noised=True, print_denoised=True)

Printing the original shape


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.263183…

Printing the noised shape


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.840095…

Printing the denoised shape


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.607093…

'10062022_212728'

#### 1. cosine similarity (latent vector)

In [22]:
def compute_cosine_similarity(denoising_dt='09302022_082805'):
    # retrieve denoised code to be compared with the original shape codes
    diffusion_model_idx = 8200
    ref_shape_idx = 94
    diffusion_model_dt = '09272022_165500'
    # denoising_dt = '09302022_082805' 
    data  = np.load(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}.npz')
    denoised_code = data['denoised_code'] # this is the already denormalized code (ie after oeprating std*denoised_code+mean)
    denoised_code = torch.tensor(denoised_code)

    # retrieve shape indices used for diffusion
    log_filename = f'./diffusion logs & models/{diffusion_model_dt}/{diffusion_model_dt}.log'

    with open(log_filename) as f:
        f = f.readlines()

    for idx, line in enumerate(f):
        if "Decoder model used" in line :
            if idx > 10:
                break
        if "Shape indices" in line:
            shape_indices = line.split('=')[1].strip().strip('[').strip(']')

    shape_indices = shape_indices.split(', ')
    shape_indices = [int(shape_index) for shape_index in shape_indices]

    # retrieve original shape codes
    decoder_dt = '09152022_190445'
    shapecode, _, shape_indices_from_decoder = get_shapecode(decoder_dt, n_shapes=6778) # specify shapecode_epoch, shape_indices, n_shapes, random_or_not if needed
    assert shape_indices == shape_indices_from_decoder


    # compute cosine similarity between denoised shape code and original shape codes
    cos_sim = []
    cos = nn.CosineSimilarity(dim=0)
    for shape_index in shape_indices:
        cos_sim.append((cos((shapecode[shape_index]),torch.squeeze(denoised_code))).detach().numpy())
                                                                 # format: 
    # 3155 cos, 
    # 6209 cos

    # saving the computed results
    cos_sim = np.array(cos_sim)
    np.savez(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}_cos_sim', cos_sim=cos_sim)
    
    return cos_sim

In [26]:
cos_sim = compute_cosine_similarity(denoising_dt='09302022_082805')
shapecode, _, shape_indices_from_decoder = get_shapecode('09152022_190445', n_shapes=6778) # specify shapecode_epoch, shape_indices, n_shapes, random_or_not if needed

for idx in (-cos_sim).argsort()[:10]:
    print(shape_indices_from_decoder[idx], cos_sim[idx])


3452 0.6196762
5310 0.59623426
2526 0.5810795
392 0.56987023
6755 0.5690398
3342 0.56885135
1926 0.5672559
5847 0.56709915
4608 0.56488675
5753 0.5642816


In [36]:
embedding = nn.Embedding(10, 3).weight.data.detach()
print(embedding)
embedding2 = embedding[torch.tensor([1, 3, 5])]
print(embedding2)
print(embedding2[1])

tensor([[ 0.6181,  0.3337, -0.4020],
        [ 0.1368, -1.2796, -0.4588],
        [ 0.8694, -0.3352, -0.2861],
        [ 0.1999, -0.8072, -1.5405],
        [-0.2542,  0.1338, -0.3113],
        [-1.5674, -0.6037, -0.5704],
        [ 0.7869, -1.0311, -0.3246],
        [-1.9719, -0.9282, -0.5351],
        [ 1.9339,  0.7259, -0.6859],
        [-0.3567,  1.1146, -0.5746]])
tensor([[ 0.1368, -1.2796, -0.4588],
        [ 0.1999, -0.8072, -1.5405],
        [-1.5674, -0.6037, -0.5704]])
tensor([ 0.1999, -0.8072, -1.5405])


In [28]:
decoder, _ = get_decoder('09152022_190445') 

for idx in (-cos_sim).argsort()[:10]:
    verts, faces = reconstruct_shapes(decoder, shapecode[shape_indices_from_decoder[idx]])
    mp.plot(verts, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.484287…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(48.222388…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.383041…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.161293…

KeyboardInterrupt: 

#### 2. Euclidean distance (latent vector)

In [82]:
# retrieve denoised code to be compared with the original shape codes
diffusion_model_idx = 8200
ref_shape_idx = 94
diffusion_model_dt = '09272022_165500'
denoising_dt = '09302022_082805' # saving_dt
data  = np.load(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}.npz')
denoised_code = data['denoised_code'] # this is the already denormalized code (ie after oeprating std*denoised_code+mean)
denoised_code = torch.tensor(denoised_code)

# retrieve shape indices used for diffusion
log_filename = f'./diffusion logs & models/{diffusion_model_dt}/{diffusion_model_dt}.log'

with open(log_filename) as f:
    f = f.readlines()

for idx, line in enumerate(f):
    if "Decoder model used" in line :
        if idx > 10:
            break
    if "Shape indices" in line:
        shape_indices = line.split('=')[1].strip().strip('[').strip(']')

shape_indices = shape_indices.split(', ')
shape_indices = [int(shape_index) for shape_index in shape_indices]

# retrieve original shape codes
decoder_dt = '09152022_190445'
shapecode, _, shape_indices_from_decoder = get_shapecode(decoder_dt, n_shapes=6778) # specify shapecode_epoch, shape_indices, n_shapes, random_or_not if needed
assert shape_indices == shape_indices_from_decoder

std = np.std(np.asarray(shapecode))
mean = np.mean(np.asarray(shapecode))
shapecode = (shapecode-mean)/std

# compute cosine similarity between denoised shape code and original shape codes
euc_dist_diff = []
cos = nn.CosineSimilarity(dim=0)
for shape_index in shape_indices:
    euc_dist_diff.append(torch.linalg.norm((std*shapecode[int(shape_index)] + mean)-torch.squeeze(denoised_code)).detach().numpy())
    
# format: 
# 3155 cos, 
# 6209 cos
    
# saving the computed results
euc_dist_diff = np.array(euc_dist_diff)
np.savez(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}_euc_dist_diff', euc_dist_diff=euc_dist_diff)

In [100]:
euc_dist_diff

print(euc_dist_diff)
for idx in euc_dist_diff.argsort()[:10]:
    print(shape_indices[idx], euc_dist_diff[idx])

[1.8496909 1.9283289 1.6956576 ... 2.0635808 1.71431   1.6427572]
3452 1.302804
6725 1.323305
5110 1.3270646
3704 1.3292283
1926 1.3684391
4560 1.3839875
4334 1.3858078
6755 1.3868244
6557 1.3869848
3342 1.3908254


In [101]:
decoder, _ = get_decoder(decoder_dt) 

for idx in euc_dist_diff.argsort()[:10]:
    verts, faces = reconstruct_shapes(decoder, shapecode[shape_indices[idx]])
    mp.plot(verts, faces)

ValueError: Surface level must be within volume data range.

#### 3. chamfer distance & absolute sdfs 

In [102]:
from scipy.spatial import cKDTree as KDTree

def compute_symmetric_chamfer_distance(verts1, verts2):
    verts1_kd_tree = KDTree(verts1)
    one_distances, one_vertex_ids = verts1_kd_tree.query(verts2)

    verts2_kd_tree = KDTree(verts2)
    two_distances, two_vertex_ids = verts2_kd_tree.query(verts1)

    return np.mean(np.square(one_distances)) + np.mean(np.square(two_distances))

In [103]:
from scipy.spatial import cKDTree as KDTree

scale = np.array([x[1] - x[0], y[1] - y[0], z[1] - z[0]])
offset = np.array([x[0], y[0], z[0]])

symmetric_chamfers = []


# load denoised vertices to be compared
diffusion_model_dt = '09272022_165500'
diffusion_model_idx = 8200
ref_shape_idx = 94
denoising_dt = '09302022_082805' # saving_dt

data  = np.load(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}.npz')
denoised_verts = data['verts']
denoised_verts = denoised_verts*scale + offset


# load vertices for shapes reconstructed using the decoder model and original shape codes
# if vertices files do not exist, compute vertices for shapes reconstructed using the decoder model and original shape codes

decoder_dt = '09152022_190445'
if not os.path.isdir(f'./models/{decoder_dt}/reconstructed'):
    os.mkdir(f'./models/{decoder_dt}/reconstructed')
    print(f'created directory: ./models/{decoder_dt}/reconstructed')

decoder, _ = get_decoder(decoder_dt) 
shapecode, _, shape_indices = get_shapecode(decoder_dt, n_shapes=6778) # specify shapecode_epoch, shape_indices, n_shapes, random_or_not if needed

std = np.std(np.asarray(shapecode))
mean = np.mean(np.asarray(shapecode))
shapecode = (shapecode-mean)/std

    
for idx in range(len(shapecode)): # note that this idx != shape_indices[idx]
    
    if not os.path.exists(f'./models/{decoder_dt}/reconstructed/{shape_indices[idx]}'):
        original_code=std*shapecode[idx]+mean

        sdfs = []
        for points in P:
            points = points.view(-1, 3)
            sdfs.append(decoder(torch.cat((points,  original_code.repeat(points.shape[0], 1)), dim=1)).detach().numpy())
        sdfs = np.concatenate(sdfs, axis=0)
        sdfs = np.resize(sdfs,  (len(x), len(y), len(z))) 
        verts, faces, _, _ = measure.marching_cubes(sdfs, 0)
        verts = verts*scale + offset
        
        np.savez(f'./models/{decoder_dt}/reconstructed/{shape_indices[idx]}', verts=verts, faces=faces)
    else:
        data = np.load(f'./models/{decoder_dt}/reconstructed/{shape_indices[idx]}')
        verts = data['verts']

    symmetric_chamfer = compute_symmetric_chamfer_distance(denoised_verts, verts)
    print(f'{shape_indices[idx]}: {symmetric_chamfer}')
    symmetric_chamfers.append(symmetric_chamfer)
    
    if (idx%10 == 0) or (idx == len(shapecode)-1):
        np.savez(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}_chamfer', symmetric_chamfers=np.array(symmetric_chamfers))

3155: 0.03548387802587052
6209: 0.02883628414720759
3445: 0.0387960786590546
331: 0.021309713795765704
2121: 0.024491355441123026
4188: 0.07527068074778859
3980: 0.05195236335998118
3317: 0.01787335579616698
6420: 0.08188897530087884
2484: 0.034361564556328175
3904: 0.07061771776579168
2933: 0.02556465409677128


KeyboardInterrupt: 

In [104]:
data = np.load('./denoising results/model:09272022_165500_8200,shape:94/09302022_082805_chamfer.npz')
sym_chamf = data['symmetric_chamfers']
for idx in (sym_chamf).argsort()[:10]:
    print(shape_indices[idx], sym_chamf[idx])

3317 0.01787335579616698
331 0.021309713795765704
2121 0.024491355441123026
6209 0.02883628414720759
2484 0.034361564556328175
3155 0.03548387802587052
3445 0.0387960786590546
3980 0.05195236335998118
3904 0.07061771776579168
4188 0.07527068074778859


In [105]:
decoder, _ = get_decoder(decoder_dt) 

for idx in (sym_chamf).argsort()[:20]:
    verts, faces = reconstruct_shapes(decoder, shapecode[shape_indices[idx]])
    mp.plot(verts, faces)

ValueError: Surface level must be within volume data range.

### Compare multiple denoising runs

In [85]:
for _ in range(10):
    denoising_dt = run_denoising()
    data  = np.load(f'./denoising results/model:09272022_165500_8200,shape:94/{denoising_dt}.npz')
    denoised_verts = data['verts']
    denoised_faces = data['faces']
    # print('denoised shape') 
    p = mp.plot(denoised_verts*scale+offset, denoised_faces)
    cos_sim = compute_cosine_similarity(denoising_dt)
    cos_sim = np.array(cos_sim)

    for i, idx in enumerate((-cos_sim).argsort()[:2]):
        verts, faces = reconstruct_shapes(decoder, shapecode[shape_indices[idx]])
        # print(f'shape most similar (cosine similarity)')
        p.add_mesh(verts*scale+offset+ np.tile([0, 0, i*2+2],(len(verts),1)), faces) #TODO: needs sacling

    p

94


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002417…

ValueError: Surface level must be within volume data range.

### Plot original shapes

In [10]:
for shape_idx in range(len(shapecode)):

    original_code=std*shapecode[shape_idx]+mean

    sdfs = []
    for points in P:
        points = points.view(-1, 3)
        sdfs.append(decoder(torch.cat((points,  original_code.repeat(points.shape[0], 1)), dim=1)).detach().numpy())
    sdfs = np.concatenate(sdfs, axis=0)
    sdfs = np.resize(sdfs, (len(x), len(y), len(z))) 
    verts, faces, normals, values = measure.marching_cubes(sdfs, 0)
    mp.plot(verts, faces)


NameError: name 'shapecode' is not defined

In [30]:
log_filename = f'./diffusion logs & models/{diffusion_model_dt}/{diffusion_model_dt}.log'
with open(log_filename) as f:
    f = f.readlines()
for line in f: 
    if "# of shapes used for diffusion" in line :
        n_shapes = int(line.split('=')[1].strip())
    if "Decoder model used" in line :
        decoder_model = line.split('=')[1].strip()


# load the decoder and shapecodes
decoder, _ = get_decoder(decoder_model) # specify decoder_epoch if needed 
shapecode, _, shape_indices_from_decoder = get_shapecode(decoder_model, n_shapes=n_shapes)

In [29]:
diffusion_model_dt='10062022_011812'

In [31]:
shapecode.shape

torch.Size([6278, 256])

In [ ]:
n_shapes